In [4]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# 1. 定义推理示例
examples = [
    {
        "question": "乾隆和曹操谁活得更久？",
        "answer": """这里是否需要跟进问题：是的。
追问：乾隆去世时几岁？
中间答案：乾隆去世时87岁。
追问：曹操去世时几岁？
中间答案：曹操去世时66岁。
所以最终答案是：乾隆"""
    },
    {
        "question": "小米手机的创始人什么时候出生？",
        "answer": """这里是否需要跟进问题：是的。
追问：小米手机的创始人是谁？
中间答案：小米手机由 雷军 创立。
跟进：雷军什么时候出生？
中间答案：雷军出生于 1969 年 12 月 16 日。
所以最终答案是：1969 年 12 月 16 日"""
    },
    {
        "question": "王医生的儿子是小明，但小明说他没有父亲。这可能吗？",
        "answer": """思考过程：
1. 关系核对：王医生是小明的父亲吗？题目只说王医生是父母中的一方。
2. 逻辑排除：如果小明说没有父亲，说明王医生不是男性。
3. 结论：可能，因为王医生是小明的母亲。
答案：可能，王医生是女性（母亲）。"""
    },
    {
        "question": "林黛玉和鲁迅谁的力气更大？",
        "answer": """思考过程：
1. 身份核对：林黛玉是《红楼梦》中的柔弱女子；鲁迅是现实中的文豪。
2. 背景分析：林黛玉体弱多病，经常咳嗽；鲁迅虽然是文人，但身体健康且经常从事体力活动（如搬运书籍）。
3. 逻辑推导：从常识判断，健康成年男性的力量通常大于长期患病的女性。
答案：鲁迅。"""
    }
]

In [5]:
example_prompt=PromptTemplate(
  input_variables=["question","answer"],template="Question:{question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:乾隆和曹操谁活得更久？
这里是否需要跟进问题：是的。
追问：乾隆去世时几岁？
中间答案：乾隆去世时87岁。
追问：曹操去世时几岁？
中间答案：曹操去世时66岁。
所以最终答案是：乾隆


In [6]:
prompt=FewShotPromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
  suffix="Question: {input}",
  input_variables=["input"],
)

print(prompt.format(input="李白和白居易谁活得更久"))

Question:乾隆和曹操谁活得更久？
这里是否需要跟进问题：是的。
追问：乾隆去世时几岁？
中间答案：乾隆去世时87岁。
追问：曹操去世时几岁？
中间答案：曹操去世时66岁。
所以最终答案是：乾隆

Question:小米手机的创始人什么时候出生？
这里是否需要跟进问题：是的。
追问：小米手机的创始人是谁？
中间答案：小米手机由 雷军 创立。
跟进：雷军什么时候出生？
中间答案：雷军出生于 1969 年 12 月 16 日。
所以最终答案是：1969 年 12 月 16 日

Question:王医生的儿子是小明，但小明说他没有父亲。这可能吗？
思考过程：
1. 关系核对：王医生是小明的父亲吗？题目只说王医生是父母中的一方。
2. 逻辑排除：如果小明说没有父亲，说明王医生不是男性。
3. 结论：可能，因为王医生是小明的母亲。
答案：可能，王医生是女性（母亲）。

Question:林黛玉和鲁迅谁的力气更大？
思考过程：
1. 身份核对：林黛玉是《红楼梦》中的柔弱女子；鲁迅是现实中的文豪。
2. 背景分析：林黛玉体弱多病，经常咳嗽；鲁迅虽然是文人，但身体健康且经常从事体力活动（如搬运书籍）。
3. 逻辑推导：从常识判断，健康成年男性的力量通常大于长期患病的女性。
答案：鲁迅。

Question: 李白和白居易谁活得更久


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

output_parser=StrOutputParser()

chain=prompt | chat | output_parser

chain.invoke({"input":"李白和白居易谁活得更久"})


'中间答案：李白84岁，白居易62岁。所以最终答案是：白居易'

In [11]:
chain.invoke("李白和白居易谁活得更久")
#模型参数太小，回答不正确

'1. 李白：李白是唐朝著名的诗人，生活在公元701年至762年间。他活了84岁，是中国古代最伟大的诗人之一。\n\n2. 白居易：白居易（772年-845年）是唐代文学家、哲学家、诗人，与李白并称“李杜”。他活了83岁，也是中国古代最著名的诗人之一。\n\n因此，最终答案是：李白活得更久。'

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

In [25]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector,MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import Chroma

example_selector=SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1
)

question="李白和白居易谁活得更久"
selected_examples=example_selector.select_examples({"question":question})
print({f"最相似的案例:{selected_examples}"})

{"最相似的案例:[{'question': '乾隆和曹操谁活得更久？', 'answer': '这里是否需要跟进问题：是的。\\n追问：乾隆去世时几岁？\\n中间答案：乾隆去世时87岁。\\n追问：曹操去世时几岁？\\n中间答案：曹操去世时66岁。\\n所以最终答案是：乾隆'}]"}


In [26]:
prompt=FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="请根据案例的方式回答问题",
  suffix="Question: {input}",
  input_variables=["input"],
)

chain.invoke("李白和白居易谁活得更久")


'李白活了76岁，而白居易活了84岁。所以最终答案是：白居易'